In [2]:
import requests
import torch
from flask import Flask, jsonify, request
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import LongformerTokenizer, LongformerForSequenceClassification

c:\Users\Vian Martinez\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the fine-tuned model and tokenizer
MODEL_DIR = "./longformer_model"
TOKENIZER_DIR = "./longformer_model"

tokenizer = LongformerTokenizer.from_pretrained(TOKENIZER_DIR, local_files_only=True)
model = LongformerForSequenceClassification.from_pretrained(MODEL_DIR, local_files_only=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

OSError: Incorrect path_or_model_id: './longformer_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LongformerForSequenceClassification.from_pretrained('./longformer_model').to(device)
tokenizer = LongformerTokenizer.from_pretrained('./longformer_model')

OSError: Incorrect path_or_model_id: './longformer_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

: 

In [ ]:
# Flask app setup
app = Flask(__name__)


In [ ]:
# API route to fetch and analyze data
@app.route('/analyze-sentiment', methods=['POST'])
def analyze_sentiment():
    # API Endpoint to fetch data
    GET_ONM_DATA_API_URL = "http://174.138.31.241:6178/online-media/compute/getArticle"

    # Request body to fetch data
    body = {
        "projectId": "c960b616-7847-472f-9e16-e47941536fd7",
        "clippingId": "00aa0060-4295-499c-b403-6d1bea15a020",
        "endDate": "2024-12-15 23:59:59",
        "startDate": "2024-12-15 00:00:00",
        "page": 1,
        "limit": 10,
        "validationList": ["unvalidated"]
    }
    
    # Fetch articles from API
    response = requests.post(GET_ONM_DATA_API_URL, json=body)
    articles = response.json()  # List of articles

    if not articles:
        return jsonify({"error": "No articles found"}), 404

    # Labels for sentiment predictions
    labels = ["negative", "neutral", "positive"]
    
    # Analyze each article
    analyzed_articles = []
    for article in articles:
        content = article.get("content", "")
        title = article.get("title", "")
        full_text = title + " " + content

        # Tokenize and prepare input
        inputs = tokenizer(full_text, truncation=True, padding=True, max_length=512, return_tensors="pt")
        inputs = {key: val.to(device) for key, val in inputs.items()}

        # Get sentiment prediction
        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)
            prediction = torch.argmax(outputs.logits, dim=1).item()
            sentiment = labels[prediction]

        # Update article with sentiment
        article["predicted_sentiment"] = sentiment
        analyzed_articles.append(article)

    # Return updated articles
    return jsonify(analyzed_articles)

In [ ]:
# Run the Flask app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)